In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
!pip install -q sklearn

In [ ]:
%tensorflow_version 2.x

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow.compat.v2.feature_column as fc

import tensorflow as tf

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [3]:
from pathlib import Path
import os
import sys

sys.path.append(str(Path(os.getcwd()).parents[1]))
from Helper_Functions import common_utils, classification_utils
directory_to_extract_to = os.getcwd()

common_utils.load_data_from_one_drive(directory_to_extract_to, "classification_paths", "diabetes")

Extracting : 100%|██████████| 1/1 [00:00<00:00, 55.10it/s]


In [4]:
data = pd.read_csv('diabetes.csv')
df = data.copy()

In [5]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
df

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [ ]:
df.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [ ]:
#Q1 = df.quantile(0.25)
#Q3 = df.quantile(0.75)
#IQR = Q3 - Q1

#data = df[~((df < (Q1 - 3 * IQR)) |(df > (Q3 + 3 * IQR))).any(axis=1)]
#data

In [ ]:
y = data.Outcome
x = data.drop('Outcome', axis = 1)

In [ ]:
x.skew()
# the variables with -0.5 < skewness < 0.5 are symmetric i.e normally distributed
# Apply Standard Scaling for normal distribution else normalization

Pregnancies                 0.901674
Glucose                     0.173754
BloodPressure              -1.843608
SkinThickness               0.109372
Insulin                     2.272251
BMI                        -0.428982
DiabetesPedigreeFunction    1.919911
Age                         1.129597
dtype: float64

In [ ]:
# Standard Scalar
scaler = StandardScaler()
scaling_features = x[["Glucose", "SkinThickness", "BMI"]]
scaler.fit(scaling_features)
scaled_x = pd.DataFrame(scaler.transform(scaling_features),columns= scaling_features.columns)

scaled_x

,Glucose,SkinThickness,BMI
0,0.848324,0.907270,0.204013
1,-1.123396,0.530902,-0.684422
2,1.943724,-1.288212,-1.103255
3,-0.998208,0.154533,-0.494043
4,0.504055,0.907270,1.409746
...,...,...,...
763,-0.622642,1.722735,0.115169
764,0.034598,0.405445,0.610154
765,0.003301,0.154533,-0.735190
766,0.159787,-1.288212,-0.240205


In [ ]:
# Normalization
minmax_scaler = MinMaxScaler()
normalize_features = x[["Pregnancies", "BloodPressure", "Insulin", "DiabetesPedigreeFunction", "Age"]]

normalized_x = minmax_scaler.fit_transform(normalize_features)
normalized_x = pd.DataFrame(normalized_x, columns = normalize_features.columns)

normalized_x

,Pregnancies,BloodPressure,Insulin,DiabetesPedigreeFunction,Age
0,0.352941,0.590164,0.000000,0.234415,0.483333
1,0.058824,0.540984,0.000000,0.116567,0.166667
2,0.470588,0.524590,0.000000,0.253629,0.183333
3,0.058824,0.540984,0.111111,0.038002,0.000000
4,0.000000,0.327869,0.198582,0.943638,0.200000
...,...,...,...,...,...
763,0.588235,0.622951,0.212766,0.039710,0.700000
764,0.117647,0.573770,0.000000,0.111870,0.100000
765,0.294118,0.590164,0.132388,0.071307,0.150000
766,0.058824,0.491803,0.000000,0.115713,0.433333


In [ ]:
preprocessed_x = pd.concat([normalized_x, scaled_x], axis=1)
preprocessed_x

,Pregnancies,Glucose,Insulin,DiabetesPedigreeFunction,Age,BloodPressure,SkinThickness,BMI
0,0.352941,0.743719,0.000000,0.234415,0.483333,0.149641,0.907270,0.204013
1,0.058824,0.427136,0.000000,0.116567,0.166667,-0.160546,0.530902,-0.684422
2,0.470588,0.919598,0.000000,0.253629,0.183333,-0.263941,-1.288212,-1.103255
3,0.058824,0.447236,0.111111,0.038002,0.000000,-0.160546,0.154533,-0.494043
4,0.000000,0.688442,0.198582,0.943638,0.200000,-1.504687,0.907270,1.409746
...,...,...,...,...,...,...,...,...
763,0.588235,0.507538,0.212766,0.039710,0.700000,0.356432,1.722735,0.115169
764,0.117647,0.613065,0.000000,0.111870,0.100000,0.046245,0.405445,0.610154
765,0.294118,0.608040,0.132388,0.071307,0.150000,0.149641,0.154533,-0.735190
766,0.058824,0.633166,0.000000,0.115713,0.433333,-0.470732,-1.288212,-0.240205


In [ ]:
# Normalization
#mean = x.mean(axis = 0)
#x -= mean

#std = x.std(axis = 0)
#x /= std

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(x, y,test_size=0.2)
X_train, X_test, y_train, y_test = train_test_split(preprocessed_x, y,test_size=0.2)

In [ ]:
print("\nX_train:\n")
print(X_train.head())
print(X_train.shape)

print("\nX_test:\n")
print(X_test.head())
print(X_test.shape)

print("\ny_train:\n")
print(y_train.head())
print(y_train.shape)

print("\ny_test:\n")
print(y_test.head())
print(y_test.shape)


X_train:

     Pregnancies   Glucose   Insulin  DiabetesPedigreeFunction       Age  \
406     0.235294  0.577889  0.000000                  0.127242  0.416667   
256     0.176471  0.557789  0.000000                  0.204526  0.150000   
630     0.411765  0.572864  0.000000                  0.279249  0.216667   
120     0.000000  0.814070  0.118203                  0.290777  0.066667   
140     0.176471  0.643216  0.000000                  0.081127  0.566667   

     BloodPressure  SkinThickness       BMI  
406       0.149641      -1.288212 -0.392508  
256      -0.677523       1.158182 -0.240205  
630      -0.263941      -1.288212 -0.582887  
120       0.356432       2.224559  2.691630  
140       0.459827      -1.288212 -1.382478  
(614, 8)

X_test:

     Pregnancies   Glucose   Insulin  DiabetesPedigreeFunction       Age  \
152     0.529412  0.783920  0.183215                  0.474381  0.350000   
599     0.058824  0.547739  0.141844                  0.140478  0.083333   
312     0

In [ ]:
# Use KNeighbours Classifier
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(X_train,y_train)
print(knn.predict(X_test))
print(knn.score(X_test, y_test))

[0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0
 0 0 0 0 0 0 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 0
 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 1 0
 0 0 0 0 0 0 0 1 0 1 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0 1 0 0 1 0 0 0
 0 0 0 0 0 0]
0.7142857142857143


In [ ]:
# Use Logistic Regression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score, classification_report

lr = LogisticRegression()
lr.fit(X_train,y_train)
y_pred = lr.predict(X_test)
print(accuracy_score(y_test,y_pred))
print(roc_auc_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

0.8051948051948052
0.7682426404995539
              precision    recall  f1-score   support

           0       0.79      0.93      0.85        95
           1       0.84      0.61      0.71        59

    accuracy                           0.81       154
   macro avg       0.82      0.77      0.78       154
weighted avg       0.81      0.81      0.80       154



In [ ]:
COLUMNS = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
                       'BMI', 'DiabetesPedigreeFunction', 'Age']

feature_columns = []

for feature_name in COLUMNS:
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

print(feature_columns)

[NumericColumn(key='Pregnancies', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Glucose', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='BloodPressure', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SkinThickness', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Insulin', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='BMI', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='DiabetesPedigreeFunction', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='Age', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


In [ ]:
def input_fn(features, labels, training=True, batch_size=256):
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(features), labels))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [ ]:
classifier = tf.estimator.DNNClassifier(
    feature_columns=feature_columns,
    # Two hidden layers of 30 and 10 nodes respectively.
    hidden_units=[20, 10],
    # The model must choose between 3 classes.
    n_classes=2)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmppw2lfr5o', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [ ]:
classifier.train(
    input_fn=lambda: input_fn(X_train, y_train, training=True),
    steps=5000)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
INFO:tensorflow:Calling model_fn.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into /tmp/tmppw2lfr5o/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 0.6984615, step = 0
INFO:tensorflow:global_step/sec: 88.1262
INFO:tensorflow:loss = 0.69766253, step = 100 (1.143 sec)
INFO:tensorflow:global_step/sec: 110.845
INFO:tensorflow:loss = 0.706012, step = 200 (0.904 sec)
INFO:tensorflow:global_step/sec

In [ ]:
eval_result = classifier.evaluate(
    input_fn=lambda: input_fn(X_test, y_test, training=False))

print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2022-03-10T14:51:52
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmppw2lfr5o/model.ckpt-5000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 2.07677s
INFO:tensorflow:Finished evaluation at 2022-03-10-14:51:54
INFO:tensorflow:Saving dict for global step 5000: accuracy = 0.66883117, accuracy_baseline = 0.6818182, auc = 0.7292517, auc_precision_recall = 0.45330203, average_loss = 0.5662848, global_step = 5000, label/mean = 0.3181818, loss = 0.5662848, precision = 0.41666666, prediction/mean = 0.36128834, recall = 0.10204082
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5000: /tmp/tmppw2lfr5o/model.ckpt-5000

Test set accuracy: 0.669



In [ ]:
def input_fn(features, batch_size=256):
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

features = ['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
                       'BMI', 'DiabetesPedigreeFunction', 'Age']
OUTCOME = [0, 1]
predict = {}

print("Please type numeric values as prompted.")
for feature in features:
  val = input(feature + ": ")

  predict[feature] = [float(val)]

predictions = classifier.predict(input_fn=lambda: input_fn(predict))
for pred_dict in predictions:
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]

    print('Prediction is "{}" ({:.1f}%)'.format(
        OUTCOME[class_id], 100 * probability))

Please type numeric values as prompted.


KeyboardInterrupt: ignored